In [16]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O
import matplotlib.pyplot as plt
%matplotlib inline

In [17]:
df_train = pd.read_pickle('../data/train_one_hot_100000.pkl')
df_train.head()

,people_id,activity_id,act_date,outcome,ppl_date,ppl_char_10,ppl_char_11,ppl_char_12,ppl_char_13,ppl_char_14,...,ppl_char_8_type 8,ppl_char_9_type 1,ppl_char_9_type 2,ppl_char_9_type 3,ppl_char_9_type 4,ppl_char_9_type 5,ppl_char_9_type 6,ppl_char_9_type 7,ppl_char_9_type 8,ppl_char_9_type 9
28933,ppl_105,act1_232992,0.236585,0,0.739167,1,1,1,1,1,...,0,0,1,0,0,0,0,0,0,0
119450,ppl_112792,act2_1963637,0.690244,0,0.568333,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1518724,ppl_352739,act2_3806874,0.151220,1,0.597500,1,0,1,0,1,...,0,0,1,0,0,0,0,0,0,0
435859,ppl_170682,act2_1643153,0.473171,0,0.229167,1,1,1,1,1,...,0,0,1,0,0,0,0,0,0,0
927526,ppl_256890,act2_3038160,0.541463,0,0.831667,1,1,1,1,1,...,0,0,1,0,0,0,0,0,0,0


### Check data label balance

In [44]:
target_count = df_train['outcome'].value_counts()
target_count

0    55416
1    44584
Name: outcome, dtype: int64

### Prepare features and labels

In [23]:
labels = np.array(df_train['outcome'])

# Remove the labels from the features
# axis 1 refers to the columns
features = df_train.drop(['people_id', 'activity_id', 'outcome'], axis = 1)

# Saving feature names for later use
feature_list = list(features.columns)

features = np.array(features)

In [24]:
print(features.shape, labels.shape)
# print(feature_list)
print(feature_list.index('ppl_char_10'))

(100000, 18562) (100000,)
2


### Model fitting with Random Forest

In [26]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor

# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(features[:5000], labels[:5000]);

### Model testing

In [32]:
df_test = pd.read_pickle('../data/valid_one_hot_10000.pkl')

labels_test = np.array(df_test['outcome'])
features_test = df_test.drop(['people_id', 'activity_id', 'outcome'], axis = 1)

feature_list_test = list(features_test.columns)

features_test = np.array(features_test)

# # Use the forest's predict method on the test data
predictions = rf.predict(features_test)

In [42]:
# Calculate the absolute errors
errors = abs(predictions - labels_test)
print('Accuracy:', 1-round(np.sum(errors)/labels_test.shape[0], 2))

Accuracy: 0.85


### Model visualization

In [30]:
# Import tools needed for visualization
from sklearn.tree import export_graphviz
import pydot

# Pull out one tree from the forest
tree = rf.estimators_[5]

# Export the image to a dot file
export_graphviz(tree, out_file = 'tree.dot', feature_names = feature_list, rounded = True, precision = 1)

# Use dot file to create a graph
(graph, ) = pydot.graph_from_dot_file('./tree.dot')

# Write graph to a png file
graph.write_png('tree.png')

In [51]:
# from sklearn.metrics import accuracy_score

# accuracy = accuracy_score(df_test['outcome'], predictions)
# print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [49]:
# from sklearn.metrics import confusion_matrix
# # from matplotlib import pyplot as plt

# conf_mat = confusion_matrix(y_true=df_test['outcome'], y_pred=predictions)
# print('Confusion matrix:\n', conf_mat)

### Feature importances

In [52]:
# Get numerical feature importances
importances = list(rf.feature_importances_)

# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]

# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: ppl_char_38          Importance: 0.53
Variable: act_date             Importance: 0.02
Variable: ppl_date             Importance: 0.02
Variable: ppl_char_6_type 2    Importance: 0.02
Variable: ppl_group_1_group 17304 Importance: 0.01
Variable: ppl_char_2_type 2    Importance: 0.01
Variable: ppl_char_6_type 1    Importance: 0.01
Variable: ppl_char_6_type 6    Importance: 0.01
Variable: ppl_char_7_type 14   Importance: 0.01
Variable: ppl_char_10          Importance: 0.0
Variable: ppl_char_11          Importance: 0.0
Variable: ppl_char_12          Importance: 0.0
Variable: ppl_char_13          Importance: 0.0
Variable: ppl_char_14          Importance: 0.0
Variable: ppl_char_15          Importance: 0.0
Variable: ppl_char_16          Importance: 0.0
Variable: ppl_char_17          Importance: 0.0
Variable: ppl_char_18          Importance: 0.0
Variable: ppl_char_19          Importance: 0.0
Variable: ppl_char_20          Importance: 0.0
Variable: ppl_char_21          Importance: 0.0
V

Variable: act_char_10_type 3966 Importance: 0.0
Variable: act_char_10_type 3967 Importance: 0.0
Variable: act_char_10_type 3968 Importance: 0.0
Variable: act_char_10_type 397 Importance: 0.0
Variable: act_char_10_type 3975 Importance: 0.0
Variable: act_char_10_type 3977 Importance: 0.0
Variable: act_char_10_type 3978 Importance: 0.0
Variable: act_char_10_type 398 Importance: 0.0
Variable: act_char_10_type 3980 Importance: 0.0
Variable: act_char_10_type 3984 Importance: 0.0
Variable: act_char_10_type 3985 Importance: 0.0
Variable: act_char_10_type 3986 Importance: 0.0
Variable: act_char_10_type 3987 Importance: 0.0
Variable: act_char_10_type 399 Importance: 0.0
Variable: act_char_10_type 3990 Importance: 0.0
Variable: act_char_10_type 3994 Importance: 0.0
Variable: act_char_10_type 3998 Importance: 0.0
Variable: act_char_10_type 4   Importance: 0.0
Variable: act_char_10_type 40  Importance: 0.0
Variable: act_char_10_type 400 Importance: 0.0
Variable: act_char_10_type 4002 Importance: 0.

Variable: act_char_10_type 869 Importance: 0.0
Variable: act_char_10_type 8690 Importance: 0.0
Variable: act_char_10_type 87  Importance: 0.0
Variable: act_char_10_type 870 Importance: 0.0
Variable: act_char_10_type 8700 Importance: 0.0
Variable: act_char_10_type 8701 Importance: 0.0
Variable: act_char_10_type 871 Importance: 0.0
Variable: act_char_10_type 8715 Importance: 0.0
Variable: act_char_10_type 8718 Importance: 0.0
Variable: act_char_10_type 872 Importance: 0.0
Variable: act_char_10_type 8727 Importance: 0.0
Variable: act_char_10_type 8728 Importance: 0.0
Variable: act_char_10_type 873 Importance: 0.0
Variable: act_char_10_type 8735 Importance: 0.0
Variable: act_char_10_type 8739 Importance: 0.0
Variable: act_char_10_type 874 Importance: 0.0
Variable: act_char_10_type 875 Importance: 0.0
Variable: act_char_10_type 8756 Importance: 0.0
Variable: act_char_10_type 8763 Importance: 0.0
Variable: act_char_10_type 8769 Importance: 0.0
Variable: act_char_10_type 8777 Importance: 0.0


Variable: ppl_group_1_group 13627 Importance: 0.0
Variable: ppl_group_1_group 13633 Importance: 0.0
Variable: ppl_group_1_group 13636 Importance: 0.0
Variable: ppl_group_1_group 13638 Importance: 0.0
Variable: ppl_group_1_group 1364 Importance: 0.0
Variable: ppl_group_1_group 13647 Importance: 0.0
Variable: ppl_group_1_group 13652 Importance: 0.0
Variable: ppl_group_1_group 13655 Importance: 0.0
Variable: ppl_group_1_group 13657 Importance: 0.0
Variable: ppl_group_1_group 13663 Importance: 0.0
Variable: ppl_group_1_group 13664 Importance: 0.0
Variable: ppl_group_1_group 13671 Importance: 0.0
Variable: ppl_group_1_group 13676 Importance: 0.0
Variable: ppl_group_1_group 13677 Importance: 0.0
Variable: ppl_group_1_group 13678 Importance: 0.0
Variable: ppl_group_1_group 13679 Importance: 0.0
Variable: ppl_group_1_group 13685 Importance: 0.0
Variable: ppl_group_1_group 13695 Importance: 0.0
Variable: ppl_group_1_group 13699 Importance: 0.0
Variable: ppl_group_1_group 1370 Importance: 0.0
Va

Variable: ppl_group_1_group 1919 Importance: 0.0
Variable: ppl_group_1_group 19194 Importance: 0.0
Variable: ppl_group_1_group 19195 Importance: 0.0
Variable: ppl_group_1_group 19197 Importance: 0.0
Variable: ppl_group_1_group 19198 Importance: 0.0
Variable: ppl_group_1_group 1920 Importance: 0.0
Variable: ppl_group_1_group 19200 Importance: 0.0
Variable: ppl_group_1_group 19207 Importance: 0.0
Variable: ppl_group_1_group 19208 Importance: 0.0
Variable: ppl_group_1_group 1921 Importance: 0.0
Variable: ppl_group_1_group 19210 Importance: 0.0
Variable: ppl_group_1_group 19216 Importance: 0.0
Variable: ppl_group_1_group 1922 Importance: 0.0
Variable: ppl_group_1_group 19221 Importance: 0.0
Variable: ppl_group_1_group 19222 Importance: 0.0
Variable: ppl_group_1_group 19223 Importance: 0.0
Variable: ppl_group_1_group 19226 Importance: 0.0
Variable: ppl_group_1_group 19227 Importance: 0.0
Variable: ppl_group_1_group 19229 Importance: 0.0
Variable: ppl_group_1_group 19231 Importance: 0.0
Vari

...

Variable: ppl_group_1_group 27907 Importance: 0.0
Variable: ppl_group_1_group 27910 Importance: 0.0
Variable: ppl_group_1_group 27911 Importance: 0.0
Variable: ppl_group_1_group 27914 Importance: 0.0
Variable: ppl_group_1_group 27915 Importance: 0.0
Variable: ppl_group_1_group 27916 Importance: 0.0
Variable: ppl_group_1_group 27917 Importance: 0.0
Variable: ppl_group_1_group 27922 Importance: 0.0
Variable: ppl_group_1_group 27923 Importance: 0.0
Variable: ppl_group_1_group 27925 Importance: 0.0
Variable: ppl_group_1_group 27926 Importance: 0.0
Variable: ppl_group_1_group 27929 Importance: 0.0
Variable: ppl_group_1_group 27933 Importance: 0.0
Variable: ppl_group_1_group 27934 Importance: 0.0
Variable: ppl_group_1_group 27936 Importance: 0.0
Variable: ppl_group_1_group 27938 Importance: 0.0
Variable: ppl_group_1_group 27940 Importance: 0.0
Variable: ppl_group_1_group 27942 Importance: 0.0
Variable: ppl_group_1_group 27943 Importance: 0.0
Variable: ppl_group_1_group 27947 Importance: 0.0


Variable: ppl_group_1_group 33404 Importance: 0.0
Variable: ppl_group_1_group 33416 Importance: 0.0
Variable: ppl_group_1_group 3342 Importance: 0.0
Variable: ppl_group_1_group 33430 Importance: 0.0
Variable: ppl_group_1_group 3344 Importance: 0.0
Variable: ppl_group_1_group 33443 Importance: 0.0
Variable: ppl_group_1_group 33444 Importance: 0.0
Variable: ppl_group_1_group 3345 Importance: 0.0
Variable: ppl_group_1_group 33452 Importance: 0.0
Variable: ppl_group_1_group 33462 Importance: 0.0
Variable: ppl_group_1_group 33466 Importance: 0.0
Variable: ppl_group_1_group 3347 Importance: 0.0
Variable: ppl_group_1_group 33488 Importance: 0.0
Variable: ppl_group_1_group 335 Importance: 0.0
Variable: ppl_group_1_group 33506 Importance: 0.0
Variable: ppl_group_1_group 3351 Importance: 0.0
Variable: ppl_group_1_group 3352 Importance: 0.0
Variable: ppl_group_1_group 33520 Importance: 0.0
Variable: ppl_group_1_group 33524 Importance: 0.0
Variable: ppl_group_1_group 33529 Importance: 0.0
Variable

Variable: ppl_group_1_group 38249 Importance: 0.0
Variable: ppl_group_1_group 3825 Importance: 0.0
Variable: ppl_group_1_group 38251 Importance: 0.0
Variable: ppl_group_1_group 38253 Importance: 0.0
Variable: ppl_group_1_group 38257 Importance: 0.0
Variable: ppl_group_1_group 38258 Importance: 0.0
Variable: ppl_group_1_group 38263 Importance: 0.0
Variable: ppl_group_1_group 38265 Importance: 0.0
Variable: ppl_group_1_group 38266 Importance: 0.0
Variable: ppl_group_1_group 38269 Importance: 0.0
Variable: ppl_group_1_group 38271 Importance: 0.0
Variable: ppl_group_1_group 38273 Importance: 0.0
Variable: ppl_group_1_group 38274 Importance: 0.0
Variable: ppl_group_1_group 38276 Importance: 0.0
Variable: ppl_group_1_group 38277 Importance: 0.0
Variable: ppl_group_1_group 38281 Importance: 0.0
Variable: ppl_group_1_group 38282 Importance: 0.0
Variable: ppl_group_1_group 38284 Importance: 0.0
Variable: ppl_group_1_group 38287 Importance: 0.0
Variable: ppl_group_1_group 38291 Importance: 0.0
V

Variable: ppl_group_1_group 45186 Importance: 0.0
Variable: ppl_group_1_group 45187 Importance: 0.0
Variable: ppl_group_1_group 45190 Importance: 0.0
Variable: ppl_group_1_group 452 Importance: 0.0
Variable: ppl_group_1_group 4520 Importance: 0.0
Variable: ppl_group_1_group 45221 Importance: 0.0
Variable: ppl_group_1_group 45224 Importance: 0.0
Variable: ppl_group_1_group 45226 Importance: 0.0
Variable: ppl_group_1_group 4523 Importance: 0.0
Variable: ppl_group_1_group 45239 Importance: 0.0
Variable: ppl_group_1_group 4524 Importance: 0.0
Variable: ppl_group_1_group 45251 Importance: 0.0
Variable: ppl_group_1_group 45272 Importance: 0.0
Variable: ppl_group_1_group 45279 Importance: 0.0
Variable: ppl_group_1_group 45283 Importance: 0.0
Variable: ppl_group_1_group 45284 Importance: 0.0
Variable: ppl_group_1_group 4529 Importance: 0.0
Variable: ppl_group_1_group 45291 Importance: 0.0
Variable: ppl_group_1_group 45303 Importance: 0.0
Variable: ppl_group_1_group 45305 Importance: 0.0
Variab

Variable: ppl_group_1_group 7209 Importance: 0.0
Variable: ppl_group_1_group 721 Importance: 0.0
Variable: ppl_group_1_group 7212 Importance: 0.0
Variable: ppl_group_1_group 7214 Importance: 0.0
Variable: ppl_group_1_group 7217 Importance: 0.0
Variable: ppl_group_1_group 7219 Importance: 0.0
Variable: ppl_group_1_group 722 Importance: 0.0
Variable: ppl_group_1_group 7220 Importance: 0.0
Variable: ppl_group_1_group 7221 Importance: 0.0
Variable: ppl_group_1_group 7222 Importance: 0.0
Variable: ppl_group_1_group 7227 Importance: 0.0
Variable: ppl_group_1_group 723 Importance: 0.0
Variable: ppl_group_1_group 7230 Importance: 0.0
Variable: ppl_group_1_group 7232 Importance: 0.0
Variable: ppl_group_1_group 7234 Importance: 0.0
Variable: ppl_group_1_group 7235 Importance: 0.0
Variable: ppl_group_1_group 7237 Importance: 0.0
Variable: ppl_group_1_group 7242 Importance: 0.0
Variable: ppl_group_1_group 7243 Importance: 0.0
Variable: ppl_group_1_group 7245 Importance: 0.0
Variable: ppl_group_1_g